# Analyze bicycle network results
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook takes the results from 03_poi_based_generation and 04_connect_clusters and calculates/analyzes a number of measures.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-07-08

## Preliminaries

### Imports and magics

In [ ]:
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import osmnx as ox
import networkx as nx
import copy
import math
from haversine import haversine, haversine_vector
import pprint
import sys
import random
pp = pprint.PrettyPrinter(indent=4)
import pickle
import csv

from functools import partial
import pyproj
from shapely.geometry import Point, Polygon
import shapely
import shapely.ops as ops
from itertools import combinations

%matplotlib inline
import pandas as pd
from matplotlib import cm

import watermark
%load_ext watermark

In [ ]:
%watermark -n -v -m -g -iv

### Parameters

In [ ]:
placeid = "paris"
prune_measure = "betweenness"
poi_source = "railwaystation" # popdensity, grid

PATH = {}
PATH["data"] = "../data/"
PATH["plots"] = "../plots/"
PATH["results"] = "../results/"

### Functions

In [ ]:
# Graph plotting preparation functions
def my_plot_reset(G, nids = False):
    reset_plot_attributes(G)
    color_nodes(G, "red", nids)
    size_nodes(G, 8, nids)

def reset_plot_attributes(G):
    """Resets node attributes for plotting.
    All black and size 0.
    """
    G.vs["color"] = "black"
    G.vs["size"] = 0
        
def color_nodes(G, color = "blue", nids = False, use_id = True):
    """Sets the color attribute of a set of nodes nids.
    """
    if nids is False:
        nids = [v.index for v in G.vs]
        use_id = False
    if use_id:
        for nid in set(nids):
            G.vs.find(id = nid)["color"] = color
    else:
        G.vs[nids]["color"] = color

def size_nodes(G, size = 5, nids = False, use_id = True):
    """Sets the size attribute of a set of nodes nids.
    """
    if nids is False:
        nids = [v.index for v in G.vs]
        use_id = False
    if use_id:
        for nid in set(nids):
            G.vs.find(id = nid)["size"] = size
    else:
        G.vs[nids]["size"] = size

def color_edges(G, color = "blue", eids = False):
    """Sets the color attribute of a set of edge nids.
    """
    if eids is False:
        G.es["color"] = color
    else:
        G.es[eids]["color"] = color
        
def width_edges(G, width = 1, eids = False):
    """Sets the width attribute of a set of edge nids.
    """
    if eids is False:
        G.es["width"] = width
    else:
        G.es[eids]["width"] = width
        
# Other functions
def round_coordinates(G, r = 7):
    for v in G.vs:
        G.vs[v.index]["x"] = round(G.vs[v.index]["x"], r)
        G.vs[v.index]["y"] = round(G.vs[v.index]["y"], r)

def mirror_y(G):
    for v in G.vs:
        y = G.vs[v.index]["y"]
        G.vs[v.index]["y"] = -y

def dist(v1,v2):
    dist = haversine((v1['x'],v1['y']),(v2['x'],v2['y']))
    return dist


def osm_to_ig(node, edge):
    """ Turns a node and edge dataframe into an igraph Graph.
    """
    
    G = ig.Graph(directed = False)

    x_coords = node['x'].tolist() 
    y_coords = node['y'].tolist()
    ids = node['osmid'].tolist()
    coords = []

    for i in range(len(x_coords)):
        G.add_vertex(x = x_coords[i], y = y_coords[i], id = ids[i])
        coords.append((x_coords[i], y_coords[i]))

    id_dict = dict(zip(G.vs['id'], np.arange(0, G.vcount()).tolist()))
    coords_dict = dict(zip(np.arange(0, G.vcount()).tolist(), coords))

    edge_list = []
    for i in range(len(edge)):
        edge_list.append([id_dict.get(edge['u'][i]), id_dict.get(edge['v'][i])])
        
    G.add_edges(edge_list)
    G.simplify()
    new_edges = G.get_edgelist()
    
    distances_list = []
    for i in range(len(new_edges)):
        distances_list.append(haversine(coords_dict.get(new_edges[i][0]), coords_dict.get(new_edges[i][1])))

    G.es()['weight'] = distances_list
    
    return G

# def ig_to_nx(G_ig):
#     """Turn an igraph graph into a networkx graph
#     """
#     # https://stackoverflow.com/questions/23235964/interface-between-networkx-and-igraph/45127940#45127940
    
#     G = nx.Graph()
#     ids = G_ig.vs['id']
#     G.add_nodes_from(ids)
#     G.add_edges_from([(ids[x.source], ids[x.target], {"weight":x["weight"]}) for x in G_ig.es])
#     return G

proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
def geodesic_point_buffer(lat, lon, km):
    # https://gis.stackexchange.com/questions/289044/creating-buffer-circle-x-kilometers-from-point-using-python
    
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return ops.transform(project, buf).exterior.coords[:]


def calculate_directness(G, indices):
    """Calculate directness on G over all pairs in nodes_sample
    """
    # To do: consider only connected components, or only the largest?
    
    poi_edges = []
    for c, v in enumerate(indices):
        poi_edges.append(G.get_shortest_paths(v, indices[c:], weights = "weight", output = "epath"))
    
    total_distance_network = 0
    for paths_e in poi_edges:
        for path_e in paths_e:
            # Sum up distances of path segments from first to last node
            total_distance_network += sum([G.es[e]['weight'] for e in path_e])
    
    comb = combinations(indices, 2)
    v1 = []
    v2 = []
    for tup in comb:
        v1.append((G.vs[tup[0]]["x"], G.vs[tup[0]]["y"]))
        v2.append((G.vs[tup[1]]["x"], G.vs[tup[1]]["y"]))
    total_distance_haversine = sum(haversine_vector(v1, v2))
    
    return total_distance_haversine / total_distance_network

## Load data

In [ ]:
node_carall = pd.read_csv(PATH["data"]+placeid+'_carall_nodes.csv')
edge_carall = pd.read_csv(PATH["data"]+placeid+'_carall_edges.csv')
G_carall = osm_to_ig(node_carall, edge_carall)

if poi_source == "railwaystation":
    with open(PATH["data"]+placeid+'_poi_railwaystation_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]

round_coordinates(G_carall)

In [ ]:
filename = placeid + '_poi_' + poi_source + "_" + prune_measure
resultfile = open(PATH["results"] + filename + ".pickle",'rb')
res = pickle.load(resultfile)
resultfile.close()
# pp.pprint(res)

## Analyze results

cost (length)  
length / carlength  
coverage  
directness  
compare with existing bike infra  

In [ ]:
# output_place is one static file for the existing city. This can be compared to the generated infrastructure.
# Make a check if this file was already generated. If not, generate it:

output_place = {"length_carall": 0,
                "length_biketrack": 0,
                "length_bikeable": 0,
                "length_biketrackcarall": 0,
                "coverage500m_carall": 0,
                "coverage500m_biketrack": 0,
                "coverage500m_bikeable": 0,
                "coverage500m_biketrackcarall": 0,
                "directness_carall": 0,
                "directness_biketrack": 0,
                "directness_bikeable": 0,
                "directness_biketrackcarall": 0,
                "poirailway": 0,
                "poirailway_coverage500m_carall": 0,
                "poirailway_coverage500m_biketrack": 0,
                "poirailway_coverage500m_bikeable": 0,
                "poirailway_coverage500m_biketrackcarall": 0,
                "components_carall": 0,
                "components_biketrack": 0,
                "components_bikeable": 0,
                "components_biketrackcarall": 0
               }
# To do.

In [ ]:
# output contains lists for all the prune_quantile values of the coresponding results
output = {"length":[],
          "coverage500m": [],
          "directness": [],
          "poirailway_coverage500m": [],
          "components": []
         }

for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
#     print(prune_quantile)
    # LENGTH
    output["length"].append(sum([e['weight'] for e in GT.es]))
    
    # COVERAGE
    # https://macwright.org/2012/10/31/gis-with-python-shapely-fiona.html
    cov = Polygon()
    for v in GT.vs:
        buf = geodesic_point_buffer(v["x"], v["y"], 0.5) # 500m
        cov = ops.unary_union([cov, Polygon(buf)])
    
    # https://gis.stackexchange.com/questions/127607/area-in-km-from-polygon-of-coordinates
    cov_area = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj('EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat_1=cov.bounds[1],
                lat_2=cov.bounds[3])),
        cov)
    output["coverage500m"].append(cov_area.area / 1000000)
    
    # POI COVERAGE
    pois_indices = set()
    for poi in nnids:
        pois_indices.add(G_carall.vs.find(id = poi).index)

    poiscovered = 0
    for poi in pois_indices:
        v = G_carall.vs[poi]
        if Point(-v["y"], v["x"]).within(cov):
            poiscovered += 1
    output["poirailway_coverage500m"].append(poiscovered)

    # COMPONENTS
    output["components"].append(len(list(GT.components())))
    
    # DIRECTNESS
    # Do this on a random subsample for speed reasons
    if len(list(GT.components())) == 1:
        nodes_sample = random.sample(list(GT.vs), min(500, len(GT.vs)))
        output["directness"].append(calculate_directness(GT, [n.index for n in nodes_sample]))
    else:
        output["directness"].append(0)
    

In [ ]:
pp.pprint(output)

In [ ]:
# fig = plt.figure(figsize=(4, 3)) # create figure object with a (width,height)
# axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
# x, y = cov.exterior.coords.xy
# patch1 = matplotlib.patches.Polygon(np.stack((np.asarray(x), np.asarray(y)), axis=-1))
# axes.add_patch(patch1)
# axes.plot()

## Write to CSV

In [ ]:
filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".csv"

with open(PATH["results"] + filename, 'w') as f:
    w = csv.writer(f)
    w.writerow(output.keys())
    w.writerows(zip(*output.values()))